In [0]:
%run ./includes/configuration

In [0]:
%run ./includes/utilities

In [0]:
import json

schema_json = 'movie ARRAY<STRING>'
movies = spark.read.option('multiline', 'true').option('inferSchema', 'true').schema(schema_json).json("/FileStore/tables/")
movies_df_raw = movies.withColumn('movies', explode(col("movie")))

print(type(movies_df_raw))

In [0]:
#Make Notebook Idempotent
#dbutils.fs.rm(bronzePath, recurse=True)

In [0]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import *

raw_movie_DF = movies_df_raw.select(explode("movie").alias("movie"))
display(raw_movie_DF)

In [0]:
# Ingestion Metadata
from pyspark.sql.functions import current_timestamp, lit
raw_movie_data_df = (raw_movie_DF
                     .select("movie",
                             lit("files.training.databricks.com").alias("datasource"),
                             current_timestamp().alias("ingesttime"),
                             lit("new").alias("status"),
                             current_timestamp().cast("date").alias("ingestdate")
                            )
                    )

In [0]:
#Partitioning
from pyspark.sql.functions import col
(raw_movie_data_df.select("datasource",
                          "ingesttime",
                          "movie",
                          "status",
                          col("ingestdate").alias("p_ingestdate"))
 .write.format("delta")
 .mode("append")
 .partitionBy("p_ingestdate")
 .save(bronzePath)
)

In [0]:
display(dbutils.fs.ls(bronzePath))

In [0]:
spark.sql("""
drop table if exists movie_bronze
""")

spark.sql(f"""
create table movie_bronze
using delta
location "{bronzePath}"
""")